In [2]:
import requests, json, re
from IPython.display import Markdown, display

LLAMA_ENDPOINT = "http://localhost:5001/llama"

# 强化系统提示：请模型只输出 A/B/C/D + 一句话理由，并用 JSON 格式，便于解析
SYSTEM_PROMPT = (
    "You are a clinical MCQ assistant for cardiovascular topics.\n"
    "- Always answer with a single letter among [A, B, C, D].\n"
    "- Then provide a one-sentence medical reasoning.\n"
    'Output strictly in JSON: {"answer":"<A|B|C|D>","reason":"<one sentence>"}.\n'
    "Do not include any other text before or after the JSON."
)

def ask_llama_mcq_api5001(question_text: str, max_tokens: int = 256, timeout=120):
    payload = {
        "system_message": SYSTEM_PROMPT,
        "user_message": question_text,
        "max_tokens": max_tokens
        # 你的服务器目前展示的字段就这三个；如果支持 temperature/top_p，再加也行
    }
    r = requests.post(LLAMA_ENDPOINT, json=payload, timeout=timeout)
    r.raise_for_status()
    data = r.json()
    raw = data["choices"][0]["text"].strip()

    # 1) 首选 JSON 解析（我们在 system 里已经强制要求 JSON）
    ans, reason = "", ""
    try:
        # 有些服务会把 <s>[INST]...[/INST] 包在前面，先尝试从字符串中提取 JSON 段
        m = re.search(r'\{.*?\}', raw, flags=re.S)
        if m:
            obj = json.loads(m.group(0))
            ans = obj.get("answer","").strip().upper()
            reason = obj.get("reason","").strip()
        else:
            raise ValueError("No JSON found")
    except Exception:
        # 2) 兜底：直接抓 A/B/C/D
        m = re.search(r"\b([ABCD])\b", raw, flags=re.I)
        ans = m.group(1).upper() if m else ""
        reason = raw

    return ans, reason, raw


In [3]:
def build_mcq_text(stem: str, A: str, B: str, C: str, D: str):
    # 清理空白，防止脏字符影响模型
    def clean(x): 
        return (x or "").strip()
    stem, A, B, C, D = map(clean, [stem, A, B, C, D])
    return f"""Question:
{stem}

Options:
A. {A}
B. {B}
C. {C}
D. {D}

Answer with a single best option.
"""


In [4]:


questions = [
    {
        "id": 1,
        "question": """Which of the following statements about Lp(a) is NOT TRUE?
A. Lp(a) is an LDL-like particle to which apo(a) is attached by disulfide linkage
B. Can be associated with an increased risk of aortic stenosis
C. Levels in an individual subject are largely determined by genetic factors
D. Can be reliably modified by diet and exercise
E. Is often the sole identifiable risk factor for premature cardiovascular disease"""
    },
    {
        "id": 2,
        "question": """Which of the following statement is not true?
A. U.K. Biobank is largest study to date in 460 000 individuals
B. Lp(a) levels measured in nanomoles per liter using an immunoturbidometric assay has excellent concordance with the WHO clinical chemistry manual
C. Median Lp(a) levels seems to be much higher in South Asians
D. Chinese have the highest prevalence of high Lp(a)
E. Lp(a) levels appear to be somewhat higher among women than among men"""
    },
    {
        "id": 3,
        "question": """Which of the following pathogenic mechanisms is true of Lp(a)?
A. Oxidized phospholipids are downregulated due to its anti-inflammatory effects
B. It is antithrombotic due to its homology with plasminogen
C. Activity of plasminogen is suppressed with high Lp(a)
D. High Lp(a) is associated with a reduced monocyte chemoattractant activity
E. Foam cell activation is reduced with high Lp(a)"""
    },
    {
        "id": 4,
        "question": """In the highest quartile of Lp(a), the magnitude of risk is highest in which of the following?
A. Heart Failure
B. Ischemic Stroke
C. Peripheral Vascular Disease
D. Myocardial Infarction
E. Calcific Aortic Stenosis"""
    },
    {
        "id": 5,
        "question": """Which of the following is TRUE regarding lipoprotein(a) management?
A. Statins often decrease lipoprotein(a)
B. Measuring lipoprotein(a) is not recommended in any established guidelines
C. Lipoprotein(a) is a highly prevalent, independent genetic risk factor for cardiovascular disease
D. Patients with elevated lipoprotein(a) have much higher true LDL-C than appreciated
E. All of the above are true"""
    },
    {
        "id": 6,
        "question": """A 58-year-old woman with a recent myocardial infarction and stent placement to the LAD 6 months ago has been tried multiple statins and has severe myalgias with high intensity and moderate intensity statins. The patient is on a maximally tolerated dose of statin (pravastatin 20 mg daily).

Ezetimibe 10 mg is then added, and lipid profile reveals: total cholesterol = 261 mg/dL; triglycerides = 150 mg/dL ; HDL = 40 mg/dL; LDL-C = 175 mg/dL; LP(a) = 200 mg/dL. What is the next best step for the reduction of future cardiovascular events?
A. Add a PCSK9 inhibitor
B. Add inclisiran
C. Add icosapent ethyl
D. Add niacin
E. Add Bempedoic acid"""
    },
    {
        "id": 7,
        "question": """A 34-year-old South Asian patient with existing ASCVD, elevated LDL-C despite optimized statin and non-statin therapy, and family history of premature ASCVD presents for follow-up. You want to test Lp(a) to see if his elevated LDL-C is caused by elevated Lp(a). Which of the following is accurate about Lp(a) testing?
A. Measure and repeat Lp(a) levels every 6 months as recommended by the guidelines
B. Preference is to use an assay that is reported in nanomoles per liter (nmol/l)
C. Preference is to use an assay that is reported in mg/dL
D. All assays are insensitive to all isoforms of Lp(a)
E. Lp(a) testing is not recommended in this setting"""
    },
    {
        "id": 8,
        "question": """Based on 2018 ACC/ AHA Cholesterol guidelines, which is TRUE regarding the threshold for measuring Lp(a) for treatment?
A. Lp(a) levels >430 nmol (>180mg/dl) are considered an ASCVD risk-enhancing factor
B. Lp(a) levels >90 nmol (>180mg/dl) are considered an ASCVD risk-enhancing factor
C. Lp(a) levels ≥125 nmol (≥50mg/dl) are considered an ASCVD risk-enhancing factor
D. Lp(a) levels ≥100 nmol (>50mg/dl) are considered an ASCVD risk-enhancing factor"""
    },
    {
        "id": 9,
        "question": """Which of the following statements is TRUE about pelacarsen, an investigative agent for Lp(a) lowering?
A. It is an Antisense Oligonucleotide that prevents production of apo(a)
B. It is a Small interference RNA that interferes mRNA translation in hepatocytes
C. It is a SiRNA interfering with biosynthesis of Lp(a ) in Liver
D. It is an oral small molecule that binds to apo(a)"""
    },
    {
        "id": 10,
        "question": """Which of the following statement regarding Lipid Apheresis is true?
A. It is typically done every 12 weeks as Lp(a) levels return to their high levels over this interval.
B. FDA approved for lowering of Lp(a ) with apheresis for Lp(a) >60 mg/dL.
C. During a single 3- to 4-hour treatment, the Lp(a) concentration is acutely lowered by 20 %.
D. In addition to lowering Lp(a), lipoprotein apheresis lowers LDL concentrations by but only by 20 %
E. Lp(a) lowering with lipoprotein apheresis do not seem to reduce the risk of ASCVD"""
    }
]

In [6]:
import json, requests, re

LLAMA_ENDPOINT = "http://localhost:5001/llama"
VALID = set("ABCDE")

SYSTEM_PROMPT = (
    "You are a clinical MCQ assistant for cardiovascular topics.\n"
    "Output ONLY one JSON: {\"answer\":\"<A|B|C|D|E>\", \"reason\":\"...\"}.\n"
    "No extra text. Answer must be A/B/C/D/E. One-sentence reason."
)

def _extract_ans_reason(raw: str):
    raw = raw.strip()
    m = re.search(r'\{.*\}', raw, flags=re.S)
    if m:
        try:
            obj = json.loads(m.group(0))
            ans = str(obj.get("answer","")).strip().upper()
            reason = str(obj.get("reason","")).strip()
            if ans in VALID and len(ans) == 1:
                return ans, reason, raw
        except Exception:
            pass
    m = re.search(r'"answer"\s*:\s*"?([A-E])"?', raw, flags=re.I)
    if m: return m.group(1).upper(), "", raw
    m = re.search(r'\b([A-E])\b', raw, flags=re.I)
    if m: return m.group(1).upper(), "", raw
    return "", "", raw

def ask_llama_api5001(question_text: str, max_tokens=128, timeout=120):
    payload = {
        "system_message": SYSTEM_PROMPT,
        "user_message": question_text,
        "max_tokens": max_tokens,
        # 可选: "temperature": 0, "top_p": 1, "seed": 42
    }
    r = requests.post(LLAMA_ENDPOINT, json=payload, timeout=timeout)
    r.raise_for_status()
    raw = r.json()["choices"][0]["text"]
    return _extract_ans_reason(raw)

# 读取刚生成的 JSONL 并跑
qs = [json.loads(line) for line in open("/Users/huangchujun/Documents/WCM/Year 1 Summer/Q&A data/10 questions/questions_eval.jsonl", encoding="utf-8")]

preds = []
for q in qs:
    ans, reason, raw = ask_llama_api5001(q["question"])
    preds.append({"id": q["id"], "pred": ans, "reason": reason, "raw": raw})
    print(f"Q{q['id']} -> {ans}: {reason}")


Q1 -> C: 
Q2 -> A: 
Q3 -> C: 
Q4 -> D: 
Q5 -> A: 
Q6 -> A: 
Q7 -> A: 
Q8 -> B: 
Q9 -> A: 
Q10 -> B: 
